# Board-Interlocking And Corporate Performance on Company Performance

Name: Bingfeng Hu
CID: 01137798 
Date: 14/06/24


Data - start with the basic data structure see if that is fine for the whole story, and edit the structure here.

Text resources use - write up workflow and graph building workflow, and grpah building here



In [1]:
import pandas as pd
import numpy as np
import networkx as nx
pd.set_option('display.max_columns', None)

import graph_analysis as ga

### 1. Introduction and Context

Describe meaningful structure and what to do here 

Which datasets to work on

What the data structure is meaningful or not

Describe using a naive model. 



Given the potential large nature of graph, for companies and directors, 
looking at sub-industry - CIK details, doing sub sampling and looking at the two years

### 2. Graph structure analysis for graph interlocking

Graph structure here - add:
1. Graph interlocking graphs
2. Graph communitiies over time
3. SVD not meaningful - maybe too sparse and some other way is needed

# 2. Review how to create graph adjacency matrices statistics for each year

1. Create the graphs by year
1. Create graph statistics by Director IDs 
1. Link Company IDs based on adjacency matrices
1. Take averages by year company id, and then link everything back

In [5]:
file_path = "raw_data/boardex_us_companies_full_data_2022_2024.csv"

boardex_data = pd.read_csv(file_path, index_col = 0)

# # Display the first few rows of the data to understand its structure

filtered_data_df = boardex_data[['boardid', 'companyid', 'directorid', 'overlapyearstart', 'overlapyearend' ]].drop_duplicates()
filtered_data_df['overlapyearend'].replace("Curr", 2024)
filtered_data_df['overlapyearend'] = pd.to_numeric(filtered_data_df['overlapyearend'], errors='coerce')

In [6]:
filtered_data_df.head()

,boardid,companyid,directorid,overlapyearstart,overlapyearend
0,401.0,2734400.0,601453.0,2020,NaN
1,401.0,3700766.0,601453.0,2023,NaN
2,401.0,61472.0,601453.0,2016,NaN
3,401.0,2055831.0,601453.0,2018,NaN
4,401.0,2065362.0,601453.0,2015,NaN


Describe the structure here

2. Create the graphs by year

Create a dictionary by year that's it, and get adjacency matrix

/tmp/ipykernel_34212/1616096872.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  graph_simplified_df['company_pair'] = graph_simplified_df.apply(
/tmp/ipykernel_34212/1616096872.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  director_pairs = graph_simplified_df.groupby('company_pair')['directorid'].apply(
/tmp/ipykernel_34212/1616096872.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

3. Calculate the graph statistics for each workflow

In [11]:
boardex_interlock_adj_dict.keys()

dict_keys([2022, 2023, 2024])

In [13]:
[x.shape for x in boardex_interlock_adj_dict.values()]

# 2024 has incomplete data

[(1694, 1694), (1059, 1059), (280, 280)]

4. Link the graph statistics back to the relevant companies

Then do averaging of these relevant statistics. If there are 0s that is fine also here.

In [50]:
yearly_graph_stats_df_lst = []

for year in [2022, 2023, 2024]:

    year_adj_matrix = boardex_interlock_adj_dict[year]

    G = nx.from_pandas_adjacency(year_adj_matrix)

    graph_density = nx.density(G) # create as a graph statistic detail

    # get the dictionary of director ids with relevant statistics
    clustering_dict = nx.clustering(G)
    degree_centrality_dict = nx.degree_centrality(G)
    betweenness_centrality_dict = nx.betweenness_centrality(G)

    df1 = pd.DataFrame.from_dict(clustering_dict, orient='index', columns=['local_clustering_coef'])
    df2 = pd.DataFrame.from_dict(degree_centrality_dict, orient='index', columns=['degree_centrality'])
    df3 = pd.DataFrame.from_dict(betweenness_centrality_dict, orient='index', columns=['betweenness_centrality'])

    # 
    combined_graph_df = pd.concat([df1, df2, df3], axis=1)

    # index is the director id, for workflow
    combined_graph_df.index.name = 'directorid'

    final_graph_df = combined_graph_df.reset_index()

    final_graph_df["graph_density"] = graph_density

    final_graph_df["year"] = year

    yearly_graph_stats_df_lst.append(final_graph_df)

Combine everything together

In [51]:
graph_stat_df_final = pd.concat(yearly_graph_stats_df_lst, axis = 0)

In [52]:
graph_stat_df_final

,directorid,local_clustering_coef,degree_centrality,betweenness_centrality,graph_density,year
0,1766630.0,1.0,0.013585,0.0,0.003216,2022
1,2777534.0,0.0,0.000591,0.0,0.003216,2022
2,2714293.0,1.0,0.002363,0.0,0.003216,2022
3,2709167.0,1.0,0.011223,0.0,0.003216,2022
4,2709141.0,1.0,0.011223,0.0,0.003216,2022
...,...,...,...,...,...,...
275,2760256.0,0.0,0.003584,0.0,0.015771,2024
276,2762116.0,1.0,0.007168,0.0,0.015771,2024
277,2764216.0,1.0,0.057348,0.0,0.015771,2024
278,2805741.0,1.0,0.007168,0.0,0.015771,2024


In [53]:
graph_stat_df_final.shape

(3033, 6)

Actual data here just has 

In [54]:
boardex_data.columns

Index(['associationtype', 'boardname', 'companyname', 'directorname',
       'overlapyearstart', 'overlapyearend', 'role', 'associatedrole',
       'conncompanyorgtype', 'boardid', 'companyid', 'directorid', 'roletitle',
       'roleboardposition', 'roleedflag', 'overlapyearstart_int',
       'overlapyearend_int', 'startcompanydatestartrole',
       'startcompanydateendrole', 'conncompanydatestartrole',
       'conncompanydateendrole', 'orgtype'],
      dtype='object')

In [55]:
boardex_data.head()

,associationtype,boardname,companyname,directorname,overlapyearstart,overlapyearend,role,associatedrole,conncompanyorgtype,boardid,companyid,directorid,roletitle,roleboardposition,roleedflag,overlapyearstart_int,overlapyearend_int,startcompanydatestartrole,startcompanydateendrole,conncompanydatestartrole,conncompanydateendrole,orgtype
0,Unlisted Org,VERICEL CORP (Aastrom Biosciences Inc prior to...,Gladius Pharmaceuticals Inc,Doctor Steve Gilman,2020,Curr,Independent Director (Brd) (SD),Scientific Advisor (Non-Brd),Private,401.0,2734400.0,601453.0,Independent Director,Brd,SD,2020,NaN,2015-01-06,NaN,NaN,NaN,Quoted
1,Unlisted Org,VERICEL CORP (Aastrom Biosciences Inc prior to...,ContraFect Corp,Doctor Steve Gilman,2023,Curr,Independent Director (Brd) (SD),Independent Vice Chairman (Brd) (SD),Private,401.0,3700766.0,601453.0,Independent Director,Brd,SD,2023,NaN,2015-01-06,NaN,2023-11-09,NaN,Quoted
2,Other,VERICEL CORP (Aastrom Biosciences Inc prior to...,Northeastern University,Doctor Steve Gilman,2016,Curr,Independent Director (Brd) (SD),Advisory Board Member,Universities,401.0,61472.0,601453.0,Independent Director,Brd,SD,2016,NaN,2015-01-06,NaN,NaN,NaN,Quoted
3,Listed Org,VERICEL CORP (Aastrom Biosciences Inc prior to...,AKEBIA THERAPEUTICS INC,Doctor Steve Gilman,2018,Curr,Independent Director (Brd) (SD),Independent Director (Brd) (SD),Quoted,401.0,2055831.0,601453.0,Independent Director,Brd,SD,2018,NaN,2015-01-06,NaN,2018-12-12,NaN,Quoted
4,Listed Org,VERICEL CORP (Aastrom Biosciences Inc prior to...,SCYNEXIS INC,Doctor Steve Gilman,2015,Curr,Independent Director (Brd) (SD),Independent Director (Brd) (SD),Quoted,401.0,2065362.0,601453.0,Independent Director,Brd,SD,2015,NaN,2015-01-06,NaN,2015-02-25,NaN,Quoted


In [56]:
boardex_data.columns

Index(['associationtype', 'boardname', 'companyname', 'directorname',
       'overlapyearstart', 'overlapyearend', 'role', 'associatedrole',
       'conncompanyorgtype', 'boardid', 'companyid', 'directorid', 'roletitle',
       'roleboardposition', 'roleedflag', 'overlapyearstart_int',
       'overlapyearend_int', 'startcompanydatestartrole',
       'startcompanydateendrole', 'conncompanydatestartrole',
       'conncompanydateendrole', 'orgtype'],
      dtype='object')

In [57]:
boardex_data[["boardid", 'boardname', "directorid", 'directorname']].shape

(88708, 4)

In [58]:
# columns is what we care about, doesn't matter since it is based on the same director so it is fine.

board_level_stats_df = pd.merge(boardex_data[["boardid", 'boardname', "directorid", 'directorname']], graph_stat_df_final, how = "inner", left_on="directorid", right_on = "directorid" )
board_level_stats_df.sort_values("year", inplace=True)

In [59]:
board_level_stats_df.shape

(67580, 9)

We have overall 615 companies, so can be useful

In [60]:
board_level_stats_df.drop_duplicates("boardid").shape

(615, 9)

In [65]:
board_level_stats_df.to_csv("final_data/company_director_level_graph_stats.csv")

Above error is due to the incorrect company matches

Get this and run by every company to get the stock id

In [69]:
board_level_stats_df.columns

Index(['boardid', 'boardname', 'directorid', 'directorname',
       'local_clustering_coef', 'degree_centrality', 'betweenness_centrality',
       'graph_density', 'year'],
      dtype='object')

In [73]:
board_director_details_df = board_level_stats_df[['boardid', 'boardname', 'local_clustering_coef', 'degree_centrality', 'betweenness_centrality',
       'graph_density', 'year']].groupby(["boardid","boardname", "year"]).mean().reset_index()

In [74]:
board_director_details_df

,boardid,boardname,year,local_clustering_coef,degree_centrality,betweenness_centrality,graph_density
0,569.0,ACCELERATE DIAGNOSTICS INC (Accelr8 Technology...,2022,1.000000,0.001181,0.000000,0.003216
1,569.0,ACCELERATE DIAGNOSTICS INC (Accelr8 Technology...,2023,1.000000,0.001890,0.000000,0.004495
2,2068.0,AMGEN INC,2022,0.428352,0.001668,0.000954,0.003216
3,2068.0,AMGEN INC,2023,0.156650,0.002396,0.001115,0.004495
4,2262.0,ANTARES PHARMA INC (De-listed 05/2022),2022,0.000000,0.000591,0.000000,0.003216
...,...,...,...,...,...,...,...
1386,3569029.0,SILO PHARMA INC,2022,0.333333,0.001772,0.000001,0.003216
1387,3569029.0,SILO PHARMA INC,2023,1.000000,0.001890,0.000000,0.004495
1388,3598524.0,COYA THERAPEUTICS INC,2022,0.372093,0.000810,0.000000,0.003216
1389,3598524.0,COYA THERAPEUTICS INC,2023,0.372093,0.001297,0.000000,0.004495


In [75]:
board_director_details_df["boardid"].drop_duplicates().count()

615

Link everything back to GVKEY to match with Compustat data

In [78]:
boardex_gvkey_df = pd.read_csv("USE_THIS_GVKEY_boardex_us_companies_keys.csv",
                               index_col= 0)
boardex_gvkey_df.head()

,permco,gvkey,companyid,score,preferred,duplicate
0,35,1259,1362.0,3.0,1.0,0.0
1,216,1602,2068.0,1.0,1.0,0.0
2,250,7799,21659.0,1.0,1.0,1.0
3,250,7799,1651568.0,1.0,1.0,1.0
4,682,18738,28834.0,1.0,1.0,0.0


In [81]:
boarded_graph_df_with_gvkey = pd.merge(boardex_gvkey_df, board_director_details_df,
         left_on = "companyid", right_on = "boardid"
         )

boarded_graph_df_with_gvkey

,permco,gvkey,companyid,score,preferred,duplicate,boardid,boardname,year,local_clustering_coef,degree_centrality,betweenness_centrality,graph_density
0,216,1602,2068.0,1.0,1.0,0.0,2068.0,AMGEN INC,2022,0.428352,0.001668,9.543488e-04,0.003216
1,216,1602,2068.0,1.0,1.0,0.0,2068.0,AMGEN INC,2023,0.156650,0.002396,1.115499e-03,0.004495
2,682,18738,28834.0,1.0,1.0,0.0,28834.0,DOMINARI HOLDINGS INC (AIkido Pharma Inc prior...,2022,0.000000,0.000852,6.180685e-07,0.003216
3,682,18738,28834.0,1.0,1.0,0.0,28834.0,DOMINARI HOLDINGS INC (AIkido Pharma Inc prior...,2023,0.000000,0.000945,0.000000e+00,0.004495
4,887,12713,2179515.0,6.0,1.0,1.0,2179515.0,ABEONA THERAPEUTICS INC (PlasmaTech Biopharmac...,2022,0.000000,0.000591,0.000000e+00,0.003216
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,59404,170430,3569029.0,1.0,1.0,0.0,3569029.0,SILO PHARMA INC,2022,0.333333,0.001772,1.396377e-06,0.003216
1411,59404,170430,3569029.0,1.0,1.0,0.0,3569029.0,SILO PHARMA INC,2023,1.000000,0.001890,0.000000e+00,0.004495
1412,59681,39018,3408934.0,3.0,1.0,0.0,3408934.0,AMBRX BIOPHARMA INC (De-listed 03/2024),2022,0.984631,0.011320,7.713779e-06,0.003216
1413,59681,39018,3408934.0,3.0,1.0,0.0,3408934.0,AMBRX BIOPHARMA INC (De-listed 03/2024),2023,1.000000,0.017958,0.000000e+00,0.004495


In [84]:
boarded_graph_df_with_gvkey = boarded_graph_df_with_gvkey.drop_duplicates(["gvkey", "year"])
boarded_graph_df_with_gvkey.shape

(1402, 13)

### 3. Analysis of Text - Annual Report for Decisioning

Average sentiment over time, and look at the graphs over time

#### Add topic details

Relevant topic changes over time, changes in relevant topics, most prominent topics - count these over time, and changes, over the few years, review give the topic structure

Add relevant functions for this, and do the relevant write up.

### 4. Regression model of Governance structures and Decisioning on Company Performance

**Maybe filter out the most relevant variables above before passing to this section here**

Describe here the naive model.

Do some basic EDA structure (correlation effects of relevant effects)

And analyse the structure here, and statistical output for the workflow here.

Convert the Stata commands here and that's it!! Put it in the same notebook!!

### 5. Conclusion and Next steps

Use README section and discuss Bayesian workflow here. Seen here features are meaningful particular on sentiment, therefore use regression to do topic modelling here.

1. Meaningful graph cluster - from EDA structure
2. Topic modelling from the regression
3. Add more data to enrich the details, given the limited interpretation, - describe features missing and add this here.
    - Add more years as needed for the workflow

Bayesian enrichment of topics - given the structure we have.